In [1]:
import csv
import logging
from logging import config

# Loading configuration file
config.fileConfig("logging.conf")

# Creates a log configuration
logger = logging.getLogger("data_ingest")

In [1]:
# import logging
# import sys


# def pysparkLogs(app_name):    
#     logger = logging.getLogger(app_name)
#     logger.setLevel(logging.INFO)
#     formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

#     stdout_handler = logging.StreamHandler(sys.stdout)
#     stdout_handler.setLevel(logging.DEBUG)
#     stdout_handler.setFormatter(formatter)

#     file_handler = logging.FileHandler('logs.log')
#     file_handler.setLevel(logging.DEBUG)
#     file_handler.setFormatter(formatter)


#     logger.addHandler(file_handler)
#     logger.addHandler(stdout_handler)
    
#     return logger

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("chapter8_monitoring") \
      .config("spark.executor.memory", '3g') \
      .config("spark.executor.cores", '3') \
      .config("spark.cores.max", '3') \
      .getOrCreate()

23/03/19 19:51:19 WARN Utils: Your hostname, EVA-01 resolves to a loopback address: 127.0.1.1; using 172.22.166.177 instead (on interface eth0)
23/03/19 19:51:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/19 19:51:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_json.show()

NameError: name 'df_json' is not defined

In [14]:
spark

In [5]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer
def _to_java_object_rdd(rdd):  
    """ Return a JavaRDD of Object by unpickling
    It will convert each Python object into Java object by Pyrolite, whenever the
    RDD is serialized in batc h or not.
    """
    rdd = rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
    return rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)

def estimate_df_size(df):
    JavaObj = _to_java_object_rdd(df.rdd)
    nbytes = spark._jvm.org.apache.spark.util.SizeEstimator.estimate(JavaObj)
    return nbytes

In [4]:
df_json = spark.read.option("multiline","true") \
                    .json('github_events.json')

23/03/19 19:51:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
estimate_df_size(df_json)

13029456

In [8]:
import os

def get_file_size(file_name, s_megabytes=False):
    file_stats = os.stat(file_name)
    if s_megabytes:
        return f"The file size in megabytes is: {file_stats.st_size / (1024 * 1024)}"
    return f"The file size in bytes is: {file_stats.st_size}"

In [9]:
get_file_size(df_json)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not DataFrame